In [ ]:
import pandas as pd
import numpy as np
# from pandas import datetime
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX


import pandas as pd
train = pd.read_csv("/kaggle/input/ue21cs342aa2/train.csv", index_col=1, parse_dates=True)
train.drop("id", axis=1, inplace=True)

In [ ]:
train

In [ ]:
test = pd.read_csv("/kaggle/input/ue21cs342aa2/test.csv", index_col=1, parse_dates=True)
test.drop("id", axis=1, inplace=True)

In [ ]:
test

In [ ]:
plt.plot(train["Close"])

Creating lags to teach how the model should learn from its past values

In [ ]:
import pandas as pd
lag_weeks_1 = 3 

train['Open_lagged1'] = train['Open'].shift(lag_weeks_1)
train['Volume_lagged1'] = train['Volume'].shift(lag_weeks_1)

In [ ]:
train

checking if the differenced values are stationary or not

In [ ]:
train_close_differenced_values = train['Close'].diff().dropna()
train_close_differenced_values.head()

Plotting acf and pacf plots, to get p,d,q values as order parameter

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# Plot the ACF (Autocorrelation Function)
plt.figure(figsize=(12, 6))
plot_acf(train_close_differenced_values,lags=40)
plt.title("ACF (Autocorrelation Function)")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.show()

In [ ]:
# Plot the PACF (Partial Autocorrelation Function)
plt.figure(figsize=(12, 6))
plot_pacf(train['Close'], lags=22)
plt.title("PACF (Partial Autocorrelation Function)")
plt.xlabel("Lag")
plt.ylabel("Partial Autocorrelation")
plt.show()

from the above graphs we can tell p=1, d=1 and q=0

In [ ]:
train_log = np.log(train["Close"])
train_log

checking if log values makes any differences

In [ ]:
train["diff"] = train.Close.diff(1)

### decomposing data and adding it to another column

# Feature Engineering

decomposing the values to check for seasonality, trend and its residulas whther or not they follow a particular pattern or not

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(train["Open"], model='additive',period=52)
open_seasonal = decomposition.seasonal
open_trend = decomposition.trend
open_resid = decomposition.resid
decomposition = seasonal_decompose(train.Volume, model='additive',period=52)
volume_seasonal = decomposition.seasonal
volume_trend = decomposition.trend
volume_resid = decomposition.resid
train["open_seasonal"]=open_seasonal
train["volume_seasonal"]=volume_seasonal
train["volume_trend"] = volume_trend
train["open_trend"] = open_trend
train["open_resid"] = open_resid
train["volume_resid"] = volume_resid

In [ ]:
train = train.interpolate(method="linear").fillna(method="bfill")
train["CloseLog"]=np.log(train.Close)

In [ ]:
train["year"] = train.index.year
train["month"] = train.index.month

In [ ]:
window_size = 10

# Calculate the SMA
train['SMA_Open'] = train['Open'].rolling(window=window_size).mean()
train['SMA_Volume'] = train['Volume'].rolling(window=window_size).mean()

In [ ]:
# Adding exponential moving average
window_size = 2

# Calculate the EMA for the 'Open' column
train['EMA_Open'] = train['Open'].ewm(span=window_size, adjust=False).mean()
train["EMA_Volume"] = train["Volume"].ewm(span=window_size, adjust=False).mean()

In [ ]:
train.dropna(inplace=True)

In [ ]:
train["Opendiff"]=train["Open"]-train["Open_lagged1"]
train["Volumediff"]=train["Volume"]-train["Volume_lagged1"]

In [ ]:
train.columns

In [ ]:
train["Open_log"] = np.log(train["Open"])

In [ ]:
train["Volume_log"] = np.log(train["Volume"])

# Best arimax search

In [ ]:
exog = sm.add_constant(train[[
        'Open_lagged1', 
        # 'Volume_lagged1',
#        'diff', 
       # 'open_seasonal', 
#        'volume_seasonal', 
       # 'volume_trend',
       # 'open_trend', 
       # 'open_resid', 
       # 'volume_resid',
       'SMA_Open', 
       # 'SMA_Volume', 
#        'EMA_Open', 
       # 'EMA_Volume', 
       # 'Opendiff',
       # 'Volumediff'
       'Open_log',
       "Volume_log"
                    ]])
endog = train.Close

after trail and error, only few columns have been chose to give better smape and better fit on the data points

In [ ]:
exog.columns

In [ ]:
import warnings
import math
warnings.filterwarnings("ignore")
import itertools
p=q=range (0,3)
d = range(0,2)
pdq = list(itertools.product(p, d, q))
store={}
for param in pdq:
    # print(param)
    try:
        model_arima = sm.tsa.arima.ARIMA(endog=endog,
                                         order=param)
        # print('hi')
        model_arima_fit = model_arima.fit()
        store[param] =  model_arima_fit.aic
    except:
        continue
sorted_dict = dict(sorted(store.items(), key=lambda item: item[1]))
print(sorted_dict)

finding p,d,q value using arima

In [ ]:
model_arima = SARIMAX(endog=endog,
                                 order=list(sorted_dict.keys())[0],
                                 seasonal_order = (0,1,1,52),
                                 exog=exog)
model_fit = model_arima.fit()
model_fit.summary()

training the model once to see how the parameters are related and the overall fit along with the AIC

In [ ]:
model_fit.resid.plot.kde()

# Checking how the model fits

In [ ]:
model_fit.fittedvalues

In [ ]:
plt.figure(figsize=(15,6))
(train.Close).plot()
(model_fit.fittedvalues).plot()
# (model_fit.fittedvalues).plot()

checking if model learn the data points

# Check how good it is, find error(smape)

In [ ]:
endog

In [ ]:
X_train = exog[:232]
y_train = endog[:232]
X_test = exog[232:]
y_test = endog[232:]

model_error_test = SARIMAX(endog=y_train,
                                    order=list(sorted_dict.keys())[0],
                                    exog=X_train,
                                    enforce_stationarity=True,
                                    seasonal_order=(0,1,1,52))
                                    
model_error_test_fit = model_error_test.fit()
forecast = model_error_test_fit.forecast(59, exog=X_test)
(y_test).plot()
(forecast).plot()

It seems to be fitting almost good

In [ ]:
smape = np.mean(np.abs((forecast) -(y_test)) / (np.abs((forecast)) + np.abs((y_test))))
smape

smape seems to be low

In [ ]:
test

In [ ]:
# test = pd.concat([train[["Open","Volume"]].tail(10),test[["Open","Volume"]]])

In [ ]:
test

## Feature engineering but for test dataset

In [ ]:
test['SMA_Open'] = test['Open'].rolling(window=10).mean().fillna(method='bfill')
test["SMA_Volume"] = test["Volume"].rolling(window=10).mean().fillna(method='bfill')

In [ ]:
# from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(test["Open"], model='additive', period=1)
open_seasonal = decomposition.seasonal.fillna(method='ffill').fillna(method='bfill')
open_trend = decomposition.trend.fillna(method='ffill').fillna(method='bfill')
open_resid = decomposition.resid.fillna(method='ffill').fillna(method='bfill')
decomposition = seasonal_decompose(test["Volume"], model='additive', period=1)
volume_seasonal = decomposition.seasonal.fillna(method='ffill').fillna(method='bfill')
volume_trend = decomposition.trend.fillna(method='ffill').fillna(method='bfill')
volume_resid = decomposition.resid.fillna(method='ffill').fillna(method='bfill')

test["open_trend"] = open_trend
test["open_seasonal"]=open_seasonal
test["open_resid"] = open_resid
test["volume_trend"]=volume_trend
test["volume_seasonal"]=volume_seasonal
test["volume_resid"] = volume_resid

In [ ]:
lag_weeks_1 = 3 
test['Open_lagged1'] = test['Open'].shift(lag_weeks_1).fillna(method='ffill').fillna(method='bfill')
test["Volume_lagged1"] = test["Volume"].shift(lag_weeks_1).fillna(method='ffill').fillna(method='bfill')
test["Opendiff"] = test["Open"] - test["Open_lagged1"]
test["Volumediff"] = test["Volume"] - test["Volume_lagged1"]

In [ ]:
test

In [ ]:
window_size = 2
test['EMA_Open'] = test['Open'].ewm(span=window_size, adjust=False).mean()
test["EMA_Volume"] = test["Volume"].ewm(span=window_size, adjust=False).mean()
test["Open_log"] = np.log(test["Open"])
test["Volume_log"] = np.log(test["Volume"])

Finally training and testing by forecasting

In [ ]:
model_error_test = SARIMAX(endog=endog,
                                    order=list(sorted_dict.keys())[0],
                                    exog=exog,
                                    enforce_stationarity=True,
                                    seasonal_order=(0,1,1,52))
                                    
model_error_test_fit = model_error_test.fit()
forecast = model_error_test_fit.forecast(100 ,exog = sm.add_constant(test[[
                              'Open_lagged1', 
                            #  'Volume_lagged1',
                     #        'diff', 
                            # 'open_seasonal', 
                     #        'volume_seasonal', 
                            # 'volume_trend',
                            # 'open_trend', 
                            # 'open_resid', 
                            # 'volume_resid',
                            'SMA_Open', 
                            # 'SMA_Volume',      
                     #        'EMA_Open', 
                            # 'EMA_Volume', 
                            # 'Opendiff',
                            # 'Volumediff'
                            'Open_log',
                            "Volume_log"
                              ]]))
plt.figure(figsize=(14,6))
(train.Close).plot()
(forecast).plot()

Graph looks good with correct forecasting

In [ ]:
test['Close'] = forecast

# Classification

In [ ]:
t1 = train
t2 = test

In [ ]:
exog

In [ ]:
test.columns, train.columns

### Using basic Decision tree classification model
Using open, close and volume to find the strategy coolumn

The strategy column in encoded into numbers to help model learn

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

X_train = train[["Open","Close","Volume"]]
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train['Strategy'])

# Initialize and train a decision tree classifier
classifier = DecisionTreeClassifier(max_depth=20,
                                    min_samples_split=2,  # Minimum samples required to split an internal node
                                    min_samples_leaf=1,
                                    criterion='gini',
                                    random_state=42)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(test[["Open","Close","Volume"]])
y_pred

In [ ]:
list(y_pred).count(0),list(y_pred).count(1),list(y_pred).count(2)

encoded output is decoded back to text

In [ ]:
ans = []
for i in range(len(y_pred)):
    if y_pred[i]== 0:
        ans.append("Buy")
    elif y_pred[i]==1:
        ans.append("Hold")
    else:
        ans.append("Sell")
y_pred=ans

Exporting the output obtained

In [ ]:
submission = pd.DataFrame()
submission["Date"] = test.index
submission["Close"] = test.Close.values
submission["Strategy"] = y_pred
submission.index.name="id"
submission.head()

In [ ]:
submission.to_csv("submission.csv",index=True)